# Tutorial 3

Here we look at indroducing some more complex components.
Speciffically, the humble diode.


### Set Up

Lets get some intital imports out the way, and set auto reload, incase we change any of the external utils etc.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

%load_ext autoreload
%autoreload 2

Now, `pyspice` imports, and setting up a logger.

In [ ]:
import PySpice
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

logger = Logging.setup_logging()

Might need to change the simulator...

In [ ]:
# # # change sim program location depending on system
# if sys.platform == "linux" or sys.platform == "linux2":
#     PySpice.Spice.Simulation.CircuitSimulator.DEFAULT_SIMULATOR = 'ngspice-subprocess'  # needed for linux
# elif sys.platform == "win32":
#     # You will get logging errors/warning, but is should work
#     pass

## Part 1: Diode

Create the circuit

In [ ]:
circuit = Circuit('Voltage Divider')

Now, lets define the 1N4148PH (Signal Diode)

In [ ]:
circuit.model('MyDiode', 'D', IS=4.352@u_nA, RS=0.6458@u_Ohm, BV=110@u_V, IBV=0.0001@u_V, N=1.906)

Add components to the circuit.

In [ ]:
circuit.V('input', 1, circuit.gnd, 10@u_V)
circuit.R(1, 1, 2, 9@u_kOhm)  # @u_kΩ is a unit of kOhms

circuit.Diode(1, 2, 3, model='MyDiode')

circuit.R(2, 3, circuit.gnd, 1@u_kOhm)
circuit.Diode(2, 3, circuit.gnd, model='MyDiode')

We can print the circuit netlist:

In [ ]:
raw_text = str(circuit)
print(raw_text)

The circuit object has some useful properties.

In [ ]:
circuit.elements

We can now create our simulation.

In [ ]:
simulator = circuit.simulator(temperature=25, nominal_temperature=25)

Then run the simulation using (in this case) and operating point analysis.

In [ ]:
analysis = simulator.operating_point()

Consider the results:

In [ ]:
analysis.nodes

In [ ]:
from utils.methods import format_analysis

results = format_analysis(analysis, cast=True)
results

## Part 2: SubCircuit

Lets create a sub sircuit to use in our main circuit.

Firstly, lets define a subcircuit consisting of a resistor and diode in parallel.

In [ ]:
from PySpice.Spice.Netlist import SubCircuit

class MySubCir(SubCircuit):

    __nodes__ = ('t_in', 't_out')
    def __init__(self, name, r=1@u_kOhm):

        SubCircuit.__init__(self, name, *self.__nodes__)

        self.R(2, 't_in', 't_out', r)
        self.Diode(2, 't_in', 't_out', model='MyDiode')

        return

Create the circuit

In [ ]:
circuit2 = Circuit('Sub Circuit Example')

Define the 1N4148PH (Signal Diode) for use

In [ ]:
circuit2.model('MyDiode', 'D', IS=4.352@u_nA, RS=0.6458@u_Ohm, BV=110@u_V, IBV=0.0001@u_V, N=1.906)

Add some components

In [ ]:
circuit2.V('input', 1, circuit2.gnd, 10@u_V)
circuit2.R(1, 1, 2, 9@u_kOhm)  # @u_kΩ is a unit of kOhms
circuit2.Diode(1, 2, 3, model='MyDiode')

Define the subcircuit for use

In [ ]:
circuit2.subcircuit(MySubCir('sub1', r=1@u_kOhm))

Finally, lets add this custome sub circuit 'component'

In [ ]:
circuit2.X(1, 'sub1', 3, circuit2.gnd)

print(circuit2)

Now, lets make a simulation and run it

In [ ]:
simulator2 = circuit2.simulator(temperature=25, nominal_temperature=25)
analysis2 = simulator2.operating_point()
format_analysis(analysis2, cast=True)